<a href="https://colab.research.google.com/github/adalbertii/modele-NLP/blob/main/Embedding_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

In [ ]:
# utworzenie prostego modelu z warstwą Embedding
model = Sequential()
embedding_layer = Embedding(input_dim=10,output_dim=4,input_length=2)
model.add(embedding_layer)
model.compile('adam','mse')

# input_dim - rozmiar słownika
# output_dim - długość wektora dla  każdego słowa
# input_length - maksymalna długość sekwencji

# W powyższym przykładzie ustawiamy 10 jako rozmiar słownictwa, ponieważ będziemy kodować liczby od 0 do 9.
# Chcemy, aby długość wektora słów wynosiła 4, stąd output_dim jest ustawione na 4.
# Długość sekwencji wejściowej do warstwy osadzania będzie wynosić 2

In [ ]:
# Teraz przekażmy przykładowe dane wejściowe do naszego modelu i zobaczmy wyniki.
input_data = np.array([[1,2]])
pred = model.predict(input_data)
print(input_data.shape)
print(pred)

1/1 [==============================] - 0s 145ms/step
(1, 2)
[[[-0.02334188  0.03078547  0.01017154  0.04552039]
  [ 0.00770851 -0.01431287  0.01345507  0.005794  ]]]


Jak widać, każde słowo (1 i 2) jest reprezentowane przez wektor o długości 4. Jeśli wydrukujemy wagi warstwy osadzania, otrzymamy poniższy wynik.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 4)              40        
                                                                 
Total params: 40 (160.00 Byte)
Trainable params: 40 (160.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
print(model.weights)

[<tf.Variable 'embedding/embeddings:0' shape=(10, 4) dtype=float32, numpy=
array([[ 0.02532529,  0.02959583, -0.02077184,  0.03307004],
       [-0.02334188,  0.03078547,  0.01017154,  0.04552039],
       [ 0.00770851, -0.01431287,  0.01345507,  0.005794  ],
       [-0.04999755,  0.01052343,  0.035213  , -0.03954636],
       [ 0.03153438,  0.0402245 ,  0.04568147, -0.03027884],
       [ 0.01681492, -0.01277689, -0.02017144,  0.00512385],
       [-0.01597724, -0.00012515, -0.02283381, -0.02160322],
       [ 0.04674724,  0.0115404 , -0.02743992,  0.00157394],
       [ 0.00702375, -0.00607336,  0.00393804, -0.04824213],
       [-0.02383035,  0.01415158,  0.03827745, -0.02038814]],
      dtype=float32)>]


Te wagi są w zasadzie reprezentacjami wektorowymi słów w słownictwie. Jak omówiliśmy wcześniej, jest to tabela odnośników o rozmiarze 10 x 4, dla słów od 0 do 9. Pierwsze słowo (0) jest reprezentowane przez pierwszy wiersz w tej tabeli, czyli

[0.02532529,  0.02959583, -0.02077184,  0.03307004]

W tym przykładzie nie wytrenowaliśmy warstwy osadzania. Wagi przypisane do wektorów słów są inicjowane losowo.

**Klasyfikacja recenzji restauracji**


Tok działań:    
*   Tokenizacja zdań na słowa.
*   Utwórzenie zakodowanego wektor\ one-hot dla każdego słowa.
*   Użycie funkcji "Padding", w celu upewnienia się, że wszystkie sekwencje mają tę samą długość.
*   Przekazanie wypełnionych sekwencji jako danych wejściowwych do warstwy Embedding.
*   Spłaszczennie danych do wktora i zastostosowanie  warstwę Dense, aby przewidywać etykietę

    
    

In [ ]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

Aby to uprościć, użyjemy łącznie 10 recenzji. Połowa z nich jest pozytywna, reprezentowana przez 0, a druga połowa jest negatywna, reprezentowana przez 1.

In [ ]:
#definicja 10 recencji
reviews =[
          'Never coming back!',
          'horrible service',
          'rude waitress',
          'cold food',
          'horrible food!',
          'awesome',
          'awesome services!',
          'rocks',
          'poor work',
          'couldn\'t have done better'
]

#definicja etykiet (1 0 negatywane, 0- pozytywna)
labels = array([1,1,1,1,1,0,0,0,0,0])

Przyjmiemy rozmiar słownictwa jako 50 i zakodujemy słowa za pomocą funkcji one_hot z Keras.

In [ ]:
Vocab_size = 50
encoded_reviews = [one_hot(d,Vocab_size) for d in reviews]
print(f'encoded reviews: {encoded_reviews}')

encoded reviews: [[43, 20, 21], [47, 4], [39, 4], [41, 8], [47, 8], [21], [21, 18], [49], [15, 7], [41, 8, 25, 6]]


Widać, że długość każdej zakodowanej recenzji jest równa liczbie słów w tej recenzji. Keras one_hot w zasadzie konwertuje każde słowo na zakodowany indeks one-hot. Teraz musimy zastosować "padduing", aby wszystkie zakodowane recenzje miały tę samą długość. Zdefiniujmy 4 jako maksymalną długość i wypełnijmy zakodowane wektory zerami na końcu.

In [ ]:
max_length = 4
padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='post')
print(padded_reviews) #Wypełnione i zakodowane recenzje będą wyglądać następująco.

[[43 20 21  0]
 [47  4  0  0]
 [39  4  0  0]
 [41  8  0  0]
 [47  8  0  0]
 [21  0  0  0]
 [21 18  0  0]
 [49  0  0  0]
 [15  7  0  0]
 [41  8 25  6]]


Po utworzeniu uzupenionej, zakodowanje  reprezentacji recenzji, jesteśmy gotowi do przekazania jej jako danych wejściowych do warstwy osadzania. W poniższym fragmencie kodu tworzymy prosty model Keras.

Ustalimy długość osadzonych wektorów dla każdego słowa na 8, a długość wejściowa będzie maksymalną długością, którą już zdefiniowaliśmy jako 4.

In [ ]:
model = Sequential()
embedding_layer = Embedding(input_dim=Vocab_size,output_dim=8,input_length=max_length)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 433 (1.69 KB)
Trainable params: 433 (1.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


**Proces uczenia zdefiniowanego modelu**

In [ ]:
model.fit(padded_reviews,labels,epochs=100,verbose=0)

Po zakończeniu treningu warstwa osadzania nauczyła się wag, które są niczym innym jak reprezentacjami wektorowymi każdego słowa. Sprawdźmy kształt macierzy wag.

In [ ]:
print(embedding_layer.get_weights()[0].shape)

(50, 8)


In [ ]:
# Jeśli sprawdzimy osadzenie dla pierwszego słowa, otrzymamy następujący wektor.
embedding_layer.get_weights()[0][0]

array([ 0.06841037, -0.13783693,  0.12766486, -0.1219708 ,  0.10954367,
       -0.07662023, -0.06978109,  0.07932504], dtype=float32)